In [10]:
import pandas as pd
import gmaps
import requests
from config import g_key

In [11]:
city_data_to_load = "weather_data/cities.csv"
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Biak,ID,2022-11-01 16:24:46,-0.9131,122.8766,76.41,86,98,2.15
1,1,Norman Wells,CA,2022-11-01 16:24:46,65.2820,-126.8329,3.24,78,100,3.44
2,2,Vaini,TO,2022-11-01 16:24:47,-21.2000,-175.2000,75.36,25,100,3.44
3,3,Rikitea,PF,2022-11-01 16:24:47,-23.1203,-134.9692,70.52,75,32,25.61
4,4,Nikolskoye,RU,2022-11-01 16:24:47,59.7035,30.7861,48.09,95,82,4.61


In [12]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [13]:
gmaps.configure(api_key=g_key)

In [14]:
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [15]:
locations = city_data_df[["Lat", "Lng"]]
max_temp = city_data_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Biak,ID,2022-11-01 16:24:46,-0.9131,122.8766,76.41,86,98,2.15
2,2,Vaini,TO,2022-11-01 16:24:47,-21.2000,-175.2000,75.36,25,100,3.44
6,6,Gainesville,US,2022-11-01 16:20:28,29.6516,-82.3248,83.07,65,0,5.75
7,7,Samarai,PG,2022-11-01 16:24:48,-10.6167,150.6667,81.14,78,15,5.59
9,9,Acari,PE,2022-11-01 16:24:49,-15.4311,-74.6158,77.76,46,34,10.56
16,16,Sakakah,SA,2022-11-01 16:24:52,29.9697,40.2064,78.37,30,96,9.42
17,17,Banjar,ID,2022-11-01 16:24:52,-8.1900,114.9675,77.27,89,94,4.38
18,18,Cape Town,ZA,2022-11-01 16:23:26,-33.9258,18.4232,82.11,41,0,17.27
20,20,Georgetown,MY,2022-11-01 16:24:54,5.4112,100.3354,81.90,85,20,3.44
26,26,Havelock,US,2022-11-01 16:24:47,34.8791,-76.9013,77.00,87,47,5.01


In [20]:
preferred_cities_df.count()

City_ID       245
City          245
Country       240
Date          245
Lat           245
Lng           245
Max Temp      245
Humidity      245
Cloudiness    245
Wind Speed    245
dtype: int64

In [21]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Biak,ID,76.41,-0.9131,122.8766,
2,Vaini,TO,75.36,-21.2000,-175.2000,
6,Gainesville,US,83.07,29.6516,-82.3248,
7,Samarai,PG,81.14,-10.6167,150.6667,
9,Acari,PE,77.76,-15.4311,-74.6158,
16,Sakakah,SA,78.37,29.9697,40.2064,
17,Banjar,ID,77.27,-8.1900,114.9675,
18,Cape Town,ZA,82.11,-33.9258,18.4232,
20,Georgetown,MY,81.90,5.4112,100.3354,
26,Havelock,US,77.00,34.8791,-76.9013,


In [22]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Biak,ID,76.41,-0.9131,122.8766,Penginapan Kim
2,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
6,Gainesville,US,83.07,29.6516,-82.3248,Stayable Select
7,Samarai,PG,81.14,-10.6167,150.6667,Nuli Sapi
9,Acari,PE,77.76,-15.4311,-74.6158,HOSPEDAJE XIMENA
16,Sakakah,SA,78.37,29.9697,40.2064,Fakhamat Al Orjoana Apartment
17,Banjar,ID,77.27,-8.1900,114.9675,The Hamsa Resort
18,Cape Town,ZA,82.11,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
20,Georgetown,MY,81.90,5.4112,100.3354,Cititel Penang
26,Havelock,US,77.00,34.8791,-76.9013,Sherwood Motel


In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [33]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))